### Import Divvy Bike Data: 

mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202201-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202202-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202203-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202204-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202205-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202206-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202207-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202208-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202209-divvy-publictripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202210-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202211-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline  202212-divvy-tripdata.csv<br>

### Import Weather Data: 
mongoimport --type csv -d chicago_bikes -c weather_daily --headerline weather_daily.csv

In [1]:
import pymongo
from pymongo import MongoClient, UpdateOne
import json

# Adding for query to find top ten stations 
#from pymongo.collection import Collection
#from pymongo.aggregation import Aggregation

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# check our list of collections
print(mongo.list_database_names())

['admin', 'autosaurus', 'chicago_bikes', 'classDB', 'config', 'epa', 'fruits_db', 'gardenDB', 'local', 'met', 'mongodbVSCodePlaygroundDB', 'petsitly_marketing', 'travel_db', 'uk_food']


In [4]:
# assign the database to a variable name
db = mongo.chicago_bikes

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['distinct_station_names', 'divvy_ridedata', 'sig_prcp_yes', 'withStationName', 'sig_prcp_yes_month', 'sig_prcp_no', 'withLatLong', 'Top10StartStations', 'Top10Routes', 'Top10EndStations', 'withoutStationName', 'divvy_rides_by_season', 'divvy_ridedata_merged', 'RouteDistance', 'weather_daily', 'divvy_rides_by_month']


In [6]:
# review a document in the customer_list collection
print(db.divvy_ridedata.find_one())

{'_id': ObjectId('652035aa8dddd3436e664723'), 'ride_id': 'C2F7DD78E82EC875', 'rideable_type': 'electric_bike', 'started_at': '2022-01-13 11:59:47', 'ended_at': '2022-01-13 12:02:44', 'start_station_name': 'Glenwood Ave & Touhy Ave', 'start_station_id': 525, 'end_station_name': 'Clark St & Touhy Ave', 'end_station_id': 'RP-007', 'start_lat': 42.0128005, 'start_lng': -87.665906, 'end_lat': 42.01256011541, 'end_lng': -87.6743671152, 'member_casual': 'casual', 'started_at_date': '2022-01-13', 'started_at_time': '11:59:47', 'ended_at_date': '2022-01-13', 'ended_at_time': '12:02:44'}


In [7]:
divvy_rides = db['divvy_ridedata']
weather_daily = db['weather_daily']
withStation = db['withStationName']
withoutStation = db['withoutStationName']

In [8]:
print(divvy_rides.count_documents({}))
print(weather_daily.count_documents({}))

5435346
365


In [23]:
documents = divvy_rides.find({})
bulk_updates = []

for document in documents:
    original_value = document["started_at"]
    parts = original_value.split(" ")
    date_part = parts[0]
    time_part = parts[1]
    bulk_updates.append(
        pymongo.UpdateOne(
            {"_id": document["_id"]},
            {
                "$set": {
                    "started_at_date": date_part,
                    "started_at_time": time_part
                }
            }
        )
    )

# Execute bulk write operations
divvy_rides.bulk_write(bulk_updates)


In [24]:
documents = divvy_rides.find({})
bulk_updates = []

for document in documents:
    original_value = document["ended_at"]
    parts = original_value.split(" ")
    date_part = parts[0]
    time_part = parts[1]
    bulk_updates.append(
        pymongo.UpdateOne(
            {"_id": document["_id"]},
            {
                "$set": {
                    "ended_at_date": date_part,
                    "ended_at_time": time_part
                }
            }
        )
    )

# Execute bulk write operations
divvy_rides.bulk_write(bulk_updates)

In [9]:
print(db.divvy_ridedata.find_one())
print(db.weather_daily.find_one())

{'_id': ObjectId('652035aa8dddd3436e664723'), 'ride_id': 'C2F7DD78E82EC875', 'rideable_type': 'electric_bike', 'started_at': '2022-01-13 11:59:47', 'ended_at': '2022-01-13 12:02:44', 'start_station_name': 'Glenwood Ave & Touhy Ave', 'start_station_id': 525, 'end_station_name': 'Clark St & Touhy Ave', 'end_station_id': 'RP-007', 'start_lat': 42.0128005, 'start_lng': -87.665906, 'end_lat': 42.01256011541, 'end_lng': -87.6743671152, 'member_casual': 'casual', 'started_at_date': '2022-01-13', 'started_at_time': '11:59:47', 'ended_at_date': '2022-01-13', 'ended_at_time': '12:02:44'}
{'_id': ObjectId('652045d1b7b2efde78396faa'), 'date': '2022-01-02', 'cloud_cover': 90.0, 'precipitation': 13.81, 'min_temp': 18.81, 'max_temp': 32.77, 'morning_temp': 28.24, 'afternoon_temp': 26.64, 'evening_temp': 19.63, 'night_temp': 32.77, 'max_windspeed': 18.41, 'prcp_inches': 0.5436997, 'sig_prcp': 'yes', 'avg_temp': 24.836666666666662}


In [10]:
divvy_rides.create_index([("started_at_date", 1)])
weather_daily.create_index([("date", 1)])


'date_1'

In [27]:
pipeline = [
    {
        '$lookup': {
            'from': 'weather_daily',
            'localField': 'started_at_date',
            'foreignField': 'date',
            'as': 'weather_data'
        }
    },
    {
        '$unwind': {
            'path': '$weather_data',
            'preserveNullAndEmptyArrays': True
        }
    },
    {
        '$merge': {
            'into': 'divvy_ridedata_merged',  # Replace with your new collection name
            'whenMatched': 'merge',  # Merge documents with matching _id fields
            'whenNotMatched': 'insert'  # Insert documents that don't match
        }
    },
]

divvy_rides.aggregate(pipeline)
print("Update completed successfully.")


Update completed successfully.


In [28]:
# Access the database and collection
collection = db["divvy_ridedata_merged"]  # Replace with your collection name

# Count the documents in the collection
document_count = collection.count_documents({})
print("Total number of documents:", document_count)


Total number of documents: 5435346


In [29]:
divvy_ridedata_merged = db['divvy_ridedata_merged']
divvy_ridedata_merged.find_one()

{'_id': ObjectId('652035aa8dddd3436e664723'),
 'end_lat': 42.01256011541,
 'end_lng': -87.6743671152,
 'end_station_id': 'RP-007',
 'end_station_name': 'Clark St & Touhy Ave',
 'ended_at': '2022-01-13 12:02:44',
 'ended_at_date': '2022-01-13',
 'ended_at_time': '12:02:44',
 'member_casual': 'casual',
 'ride_id': 'C2F7DD78E82EC875',
 'rideable_type': 'electric_bike',
 'start_lat': 42.0128005,
 'start_lng': -87.665906,
 'start_station_id': 525,
 'start_station_name': 'Glenwood Ave & Touhy Ave',
 'started_at': '2022-01-13 11:59:47',
 'started_at_date': '2022-01-13',
 'started_at_time': '11:59:47',
 'weather_data': {'_id': ObjectId('652045d1b7b2efde78396fb4'),
  'date': '2022-01-13',
  'cloud_cover': 75.0,
  'precipitation': 0.0,
  'min_temp': 30.72,
  'max_temp': 39.83,
  'morning_temp': 35.49,
  'afternoon_temp': 30.72,
  'evening_temp': 34.38,
  'night_temp': 39.22,
  'max_windspeed': 8.01}}

In [30]:
# review the collections in our new database
print(db.list_collection_names())

['divvy_ridedata', 'divvy_ridedata_merged', 'weather_daily']


In [31]:
# Use aggregation pipeline to create a collection that contains start and end station names
pipeline = [
         {"$match": {"start_station_name": {"$exists": True, "$ne": ""}, 
                     "end_station_name":{"$exists": True, "$ne": ""}}},

         {"$out": "withStationName"}
]

# Perform the aggregation
result = list(divvy_ridedata_merged.aggregate(pipeline))


In [32]:
#check to make sure that collections is updated
db.list_collection_names()

['divvy_ridedata', 'withStationName', 'divvy_ridedata_merged', 'weather_daily']

In [33]:
#check if there are any documents without start or end station names
print(db.withStationName.find_one({"start_station_name":""}))
print(db.withStationName.find_one({"end_station_name":""}))

None
None


In [34]:
# Use aggregation pipeline to create a collection that doesn't contain start and end station names
pipeline = [
         {"$match": {"start_station_name": {"$exists": True, "$eq": ""}, 
                     "end_station_name":{"$exists": True, "$eq": ""}}},

         {"$out": "withoutStationName"}
]

# Perform the aggregation
result = list(divvy_ridedata_merged.aggregate(pipeline))

In [35]:
db.list_collection_names()

['divvy_ridedata',
 'withoutStationName',
 'withStationName',
 'divvy_ridedata_merged',
 'weather_daily']

In [36]:
#check if there are any documents without start or end station names
print(db.withoutStationName.find_one({"start_station_name":{"$ne":""}}))
print(db.withoutStationName.find_one({"end_station_name":{"$ne":""}}))

None
None


In [37]:
# Use aggregation pipeline to find top ten start stations 
pipeline = [
    {
        "$group": {
            "_id": "$start_station_name",
            "count": {"$sum": 1},
            "latitude": {"$first": "$end_lat"},
            "longitude": {"$first": "$end_lng"}
        }
    },
    {
        "$sort": {"count": -1}
    },
    {
        "$limit": 10
    },
    {   "$out": "Top10StartStations"
}
]

# Perform the aggregation
result = list(withStation.aggregate(pipeline))

# Assign collection to a variable
Top10StartStations = db['Top10StartStations']

In [38]:
# Check to see that the collection was added
db.list_collection_names()

['divvy_ridedata',
 'withoutStationName',
 'withStationName',
 'Top10StartStations',
 'divvy_ridedata_merged',
 'weather_daily']

In [39]:
Top10StartStations.find_one()

{'_id': 'Streeter Dr & Grand Ave',
 'count': 68245,
 'latitude': 41.880958,
 'longitude': -87.616743}

In [40]:
# Use aggregation pipeline to find top ten end stations 
pipeline = [
    {
        "$group": {
            "_id": "$end_station_name",
            "count": {"$sum": 1},
            "latitude": {"$first": "$end_lat"},
            "longitude": {"$first": "$end_lng"}
        }
    },
    {
        "$sort": {"count": -1}
    },
    {
        "$limit": 10
    },
    {   "$out": "Top10EndStations"
}
]

# Perform the aggregation
result = list(withStation.aggregate(pipeline))

# Assign to a variable
Top10EndStations = db['Top10EndStations']

In [ ]:
# Create a pipeline query to find the top ten bike routes (by start and end station)
pipeline = [
    {
        "$group": {
            "_id": { "Start Station": "$start_station_name", "End Station": "$end_station_name"},
            "count": {"$sum": 1},
            "latitude": {"$first": "$end_lat"},
            "longitude": {"$first": "$end_lng"}
        }
    },
    {"$sort": {"count": -1}
},
    {
        "$limit": 10
},
    {   "$out": "Top10Routes"
}
]
# Perform the aggregation
result = list(withStation.aggregate(pipeline))

# Assign to a variable
Top10Routes = db['Top10Routes']